 DBName : ffb

 Tables :

   admin :
       - id      varchar(8)      primary_key
       - full_name     varchar(30)
       - user_name     varchar(15)     
       - password      varchar(8)
       - phone_number   numeric(10)
       - login_status   enum('logged in','logged out') default 'logged out'

   category :
       - category_id   varchar(8)      primary_key
       - title         varchar(10)
       - image         varchar(255)

   food :
       - food_id       varchar(8)      primary_key
       - title         varchar(50)
       - description   varchar(255)
       - price         float(10,2)
       - image         varchar(255)
       - category_id   varchar(8)      foreign_key

   user :
       - id        varchar(8)      primary_key
       - full_name      varchar(30) 
       - user_name      varchar(15)     
       - password       varchar(8)
       - phone_number   numeric(10)
       - login_status   enum('logged in','logged out') default 'logged out'

   order :
       - order_id       varchar(8)      primary_key
       - food_id        varchar(8)      foriegn_key + primary_key
       - booking_time   date
       - user_id        varchar(8)      foriegn_key
       - quantity       numeric(2)

In [1]:
import mysql.connector as sql

cnx = sql.connect(user = 'root',
                  host = 'localhost',
                  database = 'ffb')
cursor = cnx.cursor()

In [5]:
# admin :
#       - id      varchar(8)      primary_key
#       - full_name     varchar(30)
#       - user_name     varchar(15)
#       - password      varchar(8)
#       - phone_number   numeric(10)

query = """
        create table if not exists admin (
            id varchar(8) not null primary key,
            user_name     varchar(15),
            password      varchar(8),
            phone_number   numeric(10),
            login_status  enum('logged in','logged out') default 'logged out'
        )
        """

cursor.execute(query)
print('admin table created!')

admin table created!


In [6]:
# category :
#        - category_id   varchar(8)      primary_key
#        - title         varchar(10)
#        - image         varchar(255)

query = """
        create table if not exists category(
            category_id   varchar(8) not null primary key,
            title         varchar(30),
            image         varchar(255)
        )
        """

cursor.execute(query)
print('category table created!')

category table created!


In [7]:
# food :
#        - food_id       varchar(8)      primary_key
#        - title         varchar(50)
#        - description   varchar(255)
#        - price         float(10,2)
#        - image         varchar(255)
#        - category_id   varchar(8)      foreign_key

query = """
        create table if not exists food(
            food_id       varchar(8) not null primary key,
            title         varchar(50),
            description   varchar(255),
            price         float(10,2),
            image         varchar(255),
            category_id   varchar(8),   
            foreign key (category_id) references category(category_id)
        )
        """

cursor.execute(query)
print('food table created!')


food table created!


In [8]:
# user :
#        - id        varchar(8)      primary_key
#        - full_name      varchar(30) 
#        - user_name      varchar(15)
#        - password       varchar(8)
#        - phone_number   numeric(10)
#        - login_status   enum('logged in','logged out') default 'logged out'

query = """
        create table if not exists user(
            user_id        varchar(8) not null primary key,
            full_name      varchar(30),
            user_name      varchar(15),
            password       varchar(8),
            phone_number   numeric(10),
            login_status   enum('logged in','logged out') default 'logged out'
        )
        """

cursor.execute(query)
print('user table created!')


user table created!


In [9]:
# orders :
#        - order_id       varchar(8)      primary_key
#        - food_id        varchar(8)      foriegn_key + primary_key
#        - booking_time   date
#        - user_id        varchar(8)      foriegn_key
#        - quantity       numeric(2)

query = """
        create table if not exists orders(
            order_id       varchar(8) not null,
            food_id        varchar(8) not null,
            booking_time   date,
            user_id        varchar(8),
            quantity       numeric(2),
            primary key (order_id,food_id),
            foreign key (user_id) references user (user_id)
        )
        """

cursor.execute(query)
print('orders table created!')


orders table created!


In [ ]:
# Inserting Categories

query = "insert into category values(%s,%s,%s)"

data = [('cat01','Burgers',None),
        ('cat02','Sandwiches',None),
        ('cat03','Fried Chicken',None),
        ('cat04','Pizza',None),
        ('cat05','Tacos and Burritos',None),
        ('cat06','Salads',None),
        ('cat07','Sides',None),
        ('cat08','Desserts',None),
        ('cat09','Beverages',None)]

cursor.executemany(query,data)
cnx.commit()
print('Categories inserted!')


In [ ]:
# Inserting Food

query = "insert into food values(%s,%s,%s,%s,%s,%s)"

data = [('fd001','Cheese Burger','cheese burger',99.00,None,'cat01'),
        ('fd002','Veggie Burger','veggie burger',79.00,None,'cat01'),
        ('fd003','Chicken Burger','chicken burger',99.00,None,'cat01'),

        ('fd004','veg Sandwich','veg sandwich',129.00,None,'cat02'),
        ('fd005','Panner Sandwich','panner sandwich',149.00,None,'cat02'),
        ('fd006','Chicken Sandwich','chicken sandwich',99.00,None,'cat02'),

        ('fd007','Wings','wings',129.00,None,'cat03'),
        ('fd008','Drumsticks','drumsticks',179.00,None,'cat03'),
        ('fd009','Popcorn','popcorn',99.00,None,'cat03')
        ]

cursor.executemany(query,data)
cnx.commit()
print('food data inserted!')

In [13]:
from datetime import datetime

# ffb

class FFB :

    def __init__(self,cursor):
        self.cursor = cursor
        self.usertype = None
        self.ID = None
        self.username = None
        self.password = None
        
    def show_categories(self):
        query = "select category_id,title from category"
        self.cursor.execute(query)
        categories = self.cursor.fetchall()
        return categories
    
    def show_food(self,category_id):
        query = "select food_id,title,price from food where category_id = '"+category_id + "'"
        self.cursor.execute(query)
        food = self.cursor.fetchall()
        return food

    def show_food_details(self,food_id):
        query = "select * from food where food_id = '"+food_id+"'"
        self.cursor.execute(query)
        return self.cursor.fetchall()
        
    def confirm_order(self,cart):
        query = "select distinct(order_id) from orders"
        self.cursor.execute(query)
        res = self.cursor.fetchall()
        order_id = 'FFB'+str(len(res)+1).rjust(5, '0')
        data = []
        booking_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
        for food_id in cart.keys():
            data.append((order_id,food_id,booking_time,self.ID,cart[food_id]))
        query = "insert into orders values (%s,%s,%s,%s,%s)"
        self.cursor.executemany(query,data)
        cnx.commit()
        return order_id
        
    def login(self,usertype,username,password) : 
        query = "select * from "+usertype+" where user_name = %s"
        self.cursor.execute(query,[username])
        userDetails = self.cursor.fetchall()
        if len(userDetails)==0 :
            print("\nUser Not Found!")
            return 0
        if password != userDetails[0][3]:
            print("\nPassword not matched!")
            return 0
        query = "update "+usertype+" set login_status = %s where user_name= %s"
        self.cursor.execute(query,['logged in',username])
        self.usertype = usertype
        self.ID = userDetails[0][0]
        self.username = userDetails[0][2]
        self.password = userDetails[0][3]
        return 1
             
    def register(self,usertype,fullname,username,password,phone):
        query = "select * from "+usertype+" where user_name = %s"
        self.cursor.execute(query,[username])
        rows = self.cursor.fetchall()
        if  len(rows[0]) :
            print('\nUser Already Exists!')
            return 0
        query = "select * from "+usertype
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        
        id = 'ID'+str(len(rows)+1).rjust(4, '0')
        query = "insert into "+usertype+" values(%s,%s,%s,%s,%s,%s)"
        self.cursor.execute(query,[id,fullname,username,password,phone,'logged in'])
        cnx.commit()
        self.usertype = usertype
        self.ID = id
        self.username = username
        self.password = password
        return 1

In [14]:
from IPython.display import clear_output

ffb = FFB(cursor)

# Sign In / Sign Up
clear_output()
print("\nSelect an option :")
print("\n1. Log in")
print("\n2. Register")

op = input('Option : ')
clear_output()
if op == '1':
    response = 0
    while response == 0:
        usertype = input('Login Type (admin/user)?: ')
        username = input('User Name : ')
        password = input('Password : ')
        clear_output()
        response = ffb.login(usertype,username,password)
elif op == '2':
    response = 0
    while response == 0:
        usertype = input('Login Type (admin/user)?: ')
        fullname = input('Full Name : ')
        username = input('User Name : ')
        password = input('Password : ')
        phone = int(input('Phone Number : '))
        clear_output()
        response = ffb.register(usertype,fullname,username,password,phone)

clear_output()

# select food 
cart = {}
while True:
    print('\nUser Name : ',ffb.username,'\t\t\tCart : ',len(cart))
    print()
    categories_list = ffb.show_categories()

    for i in range (len(categories_list)):
        print(i+1,' ',categories_list[i][1])
    print('\nEnter 0 to proceed to payment')
    op = int(input("select the category"))
    if op == 0:
        break
    print(categories_list[op-1][0])
    clear_output()
    food = ffb.show_food(categories_list[op-1][0])
    for j in range(len(food)):
        print(j+1,' ',food[j][1],'\t',food[j][2],' /-')   
        
    op1 = int(input("enter list number to add to cart : "))
    clear_output()
    print(food[op1-1][1]," selected ")
    c = int(input("Quantity : "))
    if food[op1-1][0] in cart:
        cart[food[op1-1][0]] += c
    else:
        cart[food[op1-1][0]] = c
    clear_output()
    print("1 Add More\n2 Proceed to payment")
    op3 = int(input('Enter your choice'))
    clear_output()
    if op3 == 1:
        continue
    elif op3 == 2:
        break
    
clear_output()
print('\nUser Name : ',ffb.username)
print('_____________________________________________')
print('\n\t\tReview Order')
print('_____________________________________________\n')
print('Name\t Price\tQuantity\tTotal\n')

total = 0
for food_id in cart.keys():
    food = ffb.show_food_details(food_id)[0]
    total += cart[food_id]*float(food[3])
    print(food[1],'\t',food[3],'/- \t',cart[food_id],'\t',cart[food_id]*float(food[3]))
# cart
print('\n\t\tTotal\t\t',total,'/-')

confirm = input("confirm / cancel :")
if confirm == 'confirm':
    clear_output()
    order_id = ffb.confirm_order(cart)
    print('Order Placed Successfully!')
    print('\nOrder ID : ',order_id)
else:
    print('\n Order Canceled')

Order Placed Successfully!

Order ID :  FFB00002


In [10]:
cnx.commit()

In [15]:
# 2.
# Initialize a git repository
!git init

# Add the files to be committed
!git add .

# Commit the files
!git commit -m "Initial commit"

Initialized empty Git repository in C:/Users/Surya/Desktop/Lab Programs/DBMS/FFB/.git/


The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[master (root-commit) 60fc7cb] Initial commit
 2 files changed, 1079 insertions(+)
 create mode 100644 .ipynb_checkpoints/ffb-checkpoint.ipynb
 create mode 100644 ffb.ipynb
